In [4]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import pickle
pd.set_option('max_colwidth',100)
pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',100)

In [100]:
dir_ = "./trainning_data/corporate_attributes/"
files = os.listdir(dir_)
#读取数据，表名作为变量名
for i in files:
    if i[-3:]!='csv':
        continue
    path=dir_+i
    name=i[:-4]
    print(name+',')
    #print(path)
    cmd = "%s=pd.read_csv(path)" %name    #定义该变量要赋的值（此处为i）
    exec(cmd)
    eval("%s" %name)


company_ar_socialfee,
company_modify,
company_ar_alterstockinfo,
el_company_history_manager,
company_base_info,
company_ar_assetsinfo,
company_ar,
tax_company,
tax_qianshui,
tax_abnormal,
el_company_history_inv,
company_ar_nz,
tax_year,


In [101]:
company_base_info

,entid,honor_num,illegad_num,yea_sub,web_dum,ENTTYPE,INDUSTRYPHY,REGCAP,REGCAPCUR,RECCAP,PARNUM,LIMPARNUM,PARFORM,EXENUM,brand_num,cancel_dum,certificate_num,tax_dum
0,122697049,NaN,NaN,NaN,1,10,9,8103057.4,156.0,4700000.0,NaN,NaN,NaN,NaN,NaN,0,4.0,0
1,959830672,NaN,NaN,NaN,1,5,11,38000.0,156.0,1000.0,NaN,NaN,NaN,NaN,NaN,0,2.0,0
2,564421968,NaN,NaN,NaN,0,8,12,5000.0,156.0,5000.0,NaN,NaN,NaN,NaN,NaN,1,NaN,0
3,620868429,NaN,NaN,NaN,0,8,9,10000.0,156.0,10000.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4,1176966297,NaN,NaN,NaN,0,55,11,3000.0,156.0,3000.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12274,208371641,NaN,NaN,NaN,0,65,5,20.0,156.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12275,331374714,NaN,NaN,NaN,0,99,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12276,470500045,NaN,NaN,NaN,0,5,11,50.0,156.0,500.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12277,771018416,NaN,NaN,NaN,0,99,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [6]:
dir_ = "./trainning_data/revised_attribute/"
files = os.listdir(dir_)
#读取数据，表名作为变量名
for i in files:
    if i[-3:]!='csv':
        continue
    path=dir_+i
    name=i[:-4]
    print(name+',')
    #print(path)
    cmd = "%s=pd.read_csv(path)" %name    #定义该变量要赋的值（此处为i）
    exec(cmd)
    eval("%s" %name)


company_base_info_new,
company_ar_assetsinfo_new,
tax_year_new,


In [ ]:
################################

#   特征生成

################################

In [211]:
#company_base_info_new表特征生成
entids = company_base_info_new.entid

features = ['INDUSTRYPHY','ENTTYPE']
generated_features = {}
for feature in features:
    generated_features[feature] = []
    generated_features[feature + "_risk_exist"] = []
    generated_features[feature + "_num"] = []
    generated_features[feature + "_capital_higher"] = []
    generated_features[feature + "_paidin_capital_higher"] = []

for feature in features:
    print(f'{feature} working')
    for entid in entids:
        ent_record=company_base_info_new[company_base_info_new["entid"] == entid]
        ent_type=int(ent_record[feature])
        
        ent_sametype_record = company_base_info_new[company_base_info_new[feature] == ent_type]
        generated_features[feature].append(ent_type)
        if len(ent_sametype_record) > 0:
            
            capital_average = ent_sametype_record.REGCAP.mean()
            paidin_capital_average = ent_sametype_record.RECCAP.mean()
            
            
            generated_features[feature + "_risk_exist"].append(set(label_risk_company.entid).isdisjoint(set(ent_sametype_record.entid)))
            generated_features[feature + "_num"].append(len(ent_sametype_record))
            generated_features[feature + "_capital_higher"].append(float(ent_record['REGCAP'])>capital_average)
            generated_features[feature + "_paidin_capital_higher"].append(float(ent_record['RECCAP'])>paidin_capital_average)
            
        else:
            generated_features[feature + "_risk_exist"].append(0)
            generated_features[feature + "_num"].append(0)
            generated_features[feature + "_capital_higher"].append(0)
            generated_features[feature + "_paidin_capital_higher"].append(0)

INDUSTRYPHY working


KeyboardInterrupt: 

In [536]:
#date不要 其他直接输入
entids = company_base_info_new.entid
tax_abnormal=tax_abnormal.drop_duplicates()

features = ['tax_type','tax_state','ttype','province']


generated_features = {}
for feature in features:
    generated_features[feature] = []

for feature in features:
    for entid in entids:
        entid_record = tax_abnormal[tax_abnormal["entid"] == entid]
        
        if len(entid_record) == 1:
            generated_features[feature].append(int(entid_record[feature]))            
        else:
            generated_features[feature].append(-1)

In [103]:
#social_fee 特征生成 处理方法类似 assetinfo
entids = company_base_info_new.entid
company_ar_socialfee=company_ar_socialfee.iloc[:,:22].drop_duplicates('entid')

features=[ 'so1', 'so2', 'so3', 'so4', 'so5',
       'totalWagesSo1', 'totalWagesSo2', 'totalWagesSo3', 'totalWagesSo5',
       'totalPaymentSo1', 'totalPaymentSo2', 'totalPaymentSo3',
       'totalPaymentSo4', 'totalPaymentSo5', 'unPaidSocialInsSo1',
       'unPaidSocialInsSo2', 'unPaidSocialInsSo3', 'unPaidSocialInsSo4',
       'unPaidSocialInsSo5']

generated_features = {}
for feature in features:
    generated_features[feature] = []
    generated_features[feature + "_history_percent"] = []
    generated_features[feature + "_history_more"] = []
    generated_features[feature + "_empty"] = []

#generated_features['ispublic'] = []
generated_features['ispublic_empty'] = []

for feature in features:
    for entid in entids:
        entid_record = company_ar_socialfee[company_ar_socialfee["entid"] == entid]
        if len(entid_record) > 1:
            most_recent_record = entid_record.iloc[-1]
            history = entid_record.iloc[:-1].mean()
            generated_features[feature].append(most_recent_record[feature])
            generated_features[feature + "_history_percent"].append(most_recent_record[feature] / history[feature])
            generated_features[feature + "_history_more"].append(most_recent_record[feature] > (history[feature]*1.05))
            generated_features[feature + "_empty"].append(0)
        
        elif len(entid_record) == 1:
            most_recent_record = entid_record.iloc[-1]
            generated_features[feature].append(most_recent_record[feature])
            generated_features[feature + "_history_percent"].append(1)
            generated_features[feature + "_history_more"].append(0)
            generated_features[feature + "_empty"].append(0)
            
        else:
            generated_features[feature].append(-1)
            generated_features[feature + "_history_percent"].append(-1)
            generated_features[feature + "_history_more"].append(-1)
            generated_features[feature + "_empty"].append(1)

for entid in entids:
    entid_record = company_ar_socialfee[company_ar_socialfee["entid"] == entid]
    if len(entid_record) >= 1:
        #generated_features['ispublic'].append(entid_record.ispublic)
        generated_features['ispublic_empty'].append(0)
    else:
        #generated_features['ispublic'].append(-1)
        generated_features['ispublic_empty'].append(1)        
        


KeyboardInterrupt: 

In [104]:
company_ar_socialfee

,entid,ispublic,ANCHEYEAR,so1,so2,so3,so4,so5,totalWagesSo1,totalWagesSo2,totalWagesSo3,totalWagesSo5,totalPaymentSo1,totalPaymentSo2,totalPaymentSo3,totalPaymentSo4,totalPaymentSo5,unPaidSocialInsSo1,unPaidSocialInsSo2,unPaidSocialInsSo3,unPaidSocialInsSo4,unPaidSocialInsSo5
0,272347642,1.0,2016.0,-0.653784,-0.602713,-0.357935,1.391685,-0.049549,0.846178,-2.542993,-0.690912,-0.115746,0.849092,-0.760860,-1.857753,0.493392,-0.583225,-2.125898,-1.642271,0.294672,0.871710,-0.383763
1,432161556,1.0,2016.0,-0.883412,0.677451,0.152510,-1.034647,-1.617859,0.433976,-0.769230,0.562693,-1.611236,-1.538846,0.703442,1.525758,-0.863425,1.504182,-1.666030,-0.578373,-1.480779,-1.304366,1.405027
2,730888062,1.0,2016.0,0.260645,0.826006,0.416774,-0.489677,1.563329,-1.002218,0.950197,-1.812510,1.526011,0.988217,-2.205091,-0.230122,0.784472,-0.273433,0.028417,0.020743,-0.563894,-0.087339,1.077841
3,790901023,1.0,2016.0,3.543261,3.848212,3.247752,6.315798,5.451598,6.258752,-0.710563,0.603015,0.669738,4.148589,1.711917,0.721308,0.939664,0.892419,-0.403395,-0.045134,0.745169,0.129660,0.376188
4,208114644,1.0,2016.0,94.399780,95.629656,94.061538,94.762503,94.384371,-2.902077,0.128897,-1.989905,1.191610,-0.152666,0.139666,-1.246079,0.301653,-0.470062,-0.425677,-0.051663,1.132664,-1.001104,-0.558262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10391,208371641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10392,331374714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10393,470500045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10394,771018416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
#跨表财务指标生成
entids = company_base_info_new.entid
company_ar_assetsinfo_new=company_ar_assetsinfo_new.merge(company_base_info_new[['entid','INDUSTRYPHY']],on='entid')
tax_year_new=tax_year_new.merge(company_base_info_new[['entid','INDUSTRYPHY']],on='entid')


features = ['INDUSTRYPHY','ENTTYPE']
generated_features = {}

generated_features['A1_Interest_Coverage_Multiple_A']=[]
generated_features['A2_Interest_overage_Multiple B']=[]
generated_features['A3_Asset_liability_ratio']=[]
generated_features['A4_Ratio_of_equity_to_debt']=[]
generated_features['A5_EBITDA']=[]

generated_features['B1_equity_ratio']=[]
generated_features['B2_Profit_ratio_of_main_business']=[]
generated_features['B3_Profit_from_financial_activities']=[]
generated_features['B4_Operating_profit_ratio']=[]
generated_features['B5_Flow_rate']=[]

generated_features['C1_ROA']=[]
generated_features['C2_ROE']=[]
generated_features['C3_gross_margin']=[]
generated_features['C4_Expense_ratio_during_sales']=[]
generated_features['C5_Operating_margin_before_interest_and_tax']=[]

generated_features['D1_Financial_leverage']=[]
generated_features['D2_EM']=[]
generated_features['D3_ER']=[]


generated_features['E1_Capital_preservation_and_appreciation_rate']=[]
generated_features['E2_Growth_rate_of_total_assets']=[]
generated_features['E3_Growth_rate_of_revenue']=[]

count=0
for entid in entids:
    count+=1
    ent_indus=int(company_base_info_new[company_base_info_new.entid==entid].INDUSTRYPHY)

    asset_records=company_ar_assetsinfo_new[company_ar_assetsinfo_new.entid==entid]
    tax_records=tax_year_new[tax_year_new.entid==entid]

    ind_asset_records=company_ar_assetsinfo_new[company_ar_assetsinfo_new.INDUSTRYPHY==ent_indus].groupby('entid').mean().mean()
    ind_tax_records=tax_year_new[tax_year_new.INDUSTRYPHY==ent_indus].groupby('entid').mean().mean()

    if len(asset_records)>1:
        asset_record_0=asset_records.iloc[-1]
        asset_record_1=asset_records.iloc[-2]
    elif len(asset_records)>0:
        asset_record_0=asset_records.iloc[-1]
        asset_record_1=asset_records.iloc[-1]
    else:
        asset_record_0=ind_asset_records
        asset_record_1=ind_asset_records

    if len(tax_records)>1:
        tax_record_0=tax_records.iloc[-1]
        tax_record_1=tax_records.iloc[-2]
    elif len(tax_records)>0:
        tax_record_0=tax_records.iloc[-1]
        tax_record_1=tax_records.iloc[-1]
    else:
        tax_record_0=ind_tax_records
        tax_record_1=ind_tax_records

    temp_df=asset_record_0.append(tax_record_0)
    temp_df_1=asset_record_1.append(tax_record_1)
    
    generated_features['A1_Interest_Coverage_Multiple_A'].append(((temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.finance_expense))
    generated_features['A2_Interest_overage_Multiple B'].append(((temp_df.NETINC+temp_df.finance_expense)/temp_df.finance_expense))
    generated_features['A3_Asset_liability_ratio'].append(temp_df.LIAGRO/temp_df.ASSGRO)
    generated_features['A4_Ratio_of_equity_to_debt'].append(temp_df.TOTEQU/temp_df.LIAGRO)
    generated_features['A5_EBITDA'].append(((temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.LIAGRO))

    generated_features['B1_equity_ratio'].append(temp_df.TOTEQU/temp_df.ASSGRO)
    generated_features['B2_Profit_ratio_of_main_business'].append(((temp_df.revenue-temp_df.sales_cost)/temp_df.total_profit))
    generated_features['B3_Profit_from_financial_activities'].append(((temp_df.investment_income+temp_df.fair_value_change)/temp_df.total_profit))
    generated_features['B4_Operating_profit_ratio'].append(((temp_df.non_operating_income-temp_df.non_operating_expense)/temp_df.total_profit))
    generated_features['B5_Flow_rate'].append(temp_df.sales_tax/temp_df.revenue)

    generated_features['C1_ROA'].append(temp_df.NETINC/temp_df.ASSGRO)
    generated_features['C2_ROE'].append(temp_df.NETINC/temp_df.TOTEQU)
    generated_features['C3_gross_margin'].append(((temp_df.revenue-temp_df.sales_cost)/temp_df.revenue))
    generated_features['C4_Expense_ratio_during_sales'].append(((temp_df.sales_expense+temp_df.finance_expense+temp_df.G_expense)/temp_df.revenue))
    generated_features['C5_Operating_margin_before_interest_and_tax'].append((temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/temp_df.revenue)

    generated_features['D1_Financial_leverage'].append(((temp_df.NETINC+temp_df.incometax_taxable+temp_df.finance_expense)/(temp_df.NETINC+temp_df.incometax_taxable)))
    generated_features['D2_EM'].append(temp_df.ASSGRO/temp_df.TOTEQU)
    generated_features['D3_ER'].append(temp_df.LIAGRO/temp_df.TOTEQU)


    generated_features['E1_Capital_preservation_and_appreciation_rate'].append(temp_df.TOTEQU/temp_df_1.TOTEQU)
    generated_features['E2_Growth_rate_of_total_assets'].append(((temp_df.assets-temp_df_1.assets)/temp_df_1.assets))
    generated_features['E3_Growth_rate_of_revenue'].append(((temp_df_1.revenue-temp_df.revenue)/temp_df.revenue))
    
    #print(f'{entid}处理完毕')
    #if count>8:break

AttributeError: 'DataFrame' object has no attribute 'INDUSTRYPHY'

In [78]:
#tax_year 交叉特征
entids = company_base_info_new.entid
company_ar_assetsinfo_new=company_ar_assetsinfo_new.merge(company_base_info_new[['entid','INDUSTRYPHY']],on='entid')
tax_year_new=tax_year_new.merge(company_base_info_new[['entid','INDUSTRYPHY']],on='entid')

generated_features = {}

generated_features['gross_income'] = []
generated_features['debt_income_rate']=[]
generated_features['other_gross_rate']=[]
generated_features['main_gross_rate']=[]
generated_features['yea_sub_exsit']=[]
generated_features['adjust_gross_income']=[]


for entid in entids:
    
    #该企业所属行业
    ent_indus=int(company_base_info_new[company_base_info_new.entid==entid].INDUSTRYPHY)
    
    #是否有政府补贴
    yea_sub_exsit=(company_base_info.yea_sub[company_base_info.entid==entid]>0).iloc[0]

    #该企业数据
    asset_records=company_ar_assetsinfo_new[company_ar_assetsinfo_new.entid==entid]
    tax_records=tax_year_new[tax_year_new.entid==entid]
    
    #行业数据均值（用于填充空值）
    ind_asset_records=company_ar_assetsinfo_new[company_ar_assetsinfo_new.INDUSTRYPHY==ent_indus].groupby('entid').mean().mean()
    ind_tax_records=tax_year_new[tax_year_new.INDUSTRYPHY==ent_indus].groupby('entid').mean().mean()
    
    #确定用于计算的数据，若没有则以行业均值填充
    if(len(asset_records)>0):
        asset_record=asset_records.iloc[-1]
    else:
        asset_record=ind_asset_records
        
    if(len(tax_records)>0):
        tax_record=tax_records.iloc[-1]
    else:
        tax_record=ind_tax_records
    
    temp=asset_record.append(tax_record)
    
    #计算特征数值
    gross_income=(temp.revenue+temp.fair_value_change+temp.non_operating_income
                  +temp.main_business_income+temp.other_business_income+temp.income_tax_adjuste)

    debt_income_rate=gross_income/temp.LIAGRO

    other_gross_rate=temp.other_business_income/gross_income
    
    main_gross_rate=temp.main_business_income/gross_income

    adjust_gross_income=(-1/2*yea_sub_exsit+1)*gross_income
    
    #放入字典
    generated_features['gross_income'].append(gross_income)
    generated_features['debt_income_rate'].append(debt_income_rate)
    generated_features['other_gross_rate'].append(other_gross_rate)
    generated_features['main_gross_rate'].append(main_gross_rate)
    generated_features['yea_sub_exsit'].append(yea_sub_exsit)
    generated_features['adjust_gross_income'].append(adjust_gross_income)

In [61]:
(company_base_info.yea_sub[company_base_info.entid==entid]>0).iloc[0]

False

In [115]:
# 是否为空特征生成

entids = company_base_info_new.entid
generated_features = {}


table_list=[company_ar_socialfee,
            company_modify,
            company_ar_alterstockinfo,
            el_company_history_manager,
            company_base_info,
            company_ar_assetsinfo,
            company_ar,
            tax_company,
            tax_qianshui,
            tax_abnormal,
            el_company_history_inv,
            company_ar_nz,
            tax_year,]

count=0
for table in table_list:
    count+=1
    for feature in table.columns:
        if feature=='entid':continue
        generated_features[str(count)+'_'+feature+'_isna']=[]
        
        
for endit in entids:
    count=0
    for table in table_list:
        entid_record = table[table['entid']==entid]
        count+=1
        for feature in table.columns:
            if feature=='entid':continue
            if(len(entid_record)>0):           
                generated_features[str(count)+'_'+feature+'_isna'].append(0)
            else:
                generated_features[str(count)+'_'+feature+'_isna'].append(1)

In [113]:
generated_features

{'1_entid_isna': [],
 '1_ispublic_isna': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
######################################################

#特征存储

######################################################

In [118]:
features_frame = pd.DataFrame(generated_features)
list(features_frame.columns)
# mean=features_frame.debt_income_rate.mean()
# features_frame.debt_income_rate.fillna(mean,inplace=True)
# features_frame.isna().sum()

['1_ispublic_isna',
 '1_ANCHEYEAR_isna',
 '1_so1_isna',
 '1_so2_isna',
 '1_so3_isna',
 '1_so4_isna',
 '1_so5_isna',
 '1_totalWagesSo1_isna',
 '1_totalWagesSo2_isna',
 '1_totalWagesSo3_isna',
 '1_totalWagesSo5_isna',
 '1_totalPaymentSo1_isna',
 '1_totalPaymentSo2_isna',
 '1_totalPaymentSo3_isna',
 '1_totalPaymentSo4_isna',
 '1_totalPaymentSo5_isna',
 '1_unPaidSocialInsSo1_isna',
 '1_unPaidSocialInsSo2_isna',
 '1_unPaidSocialInsSo3_isna',
 '1_unPaidSocialInsSo4_isna',
 '1_unPaidSocialInsSo5_isna',
 '2_ALTDATE_isna',
 '2_ALTTIME_isna',
 '3_ANCHEYEAR_isna',
 '3_INV_isna',
 '3_TRANSAMPR_isna',
 '3_TRANSAMAFT_isna',
 '3_ALTDATE_isna',
 '4_NAME_isna',
 '4_indate_isna',
 '4_outdate_isna',
 '5_honor_num_isna',
 '5_illegad_num_isna',
 '5_yea_sub_isna',
 '5_web_dum_isna',
 '5_ENTTYPE_isna',
 '5_INDUSTRYPHY_isna',
 '5_REGCAP_isna',
 '5_REGCAPCUR_isna',
 '5_RECCAP_isna',
 '5_PARNUM_isna',
 '5_LIMPARNUM_isna',
 '5_PARFORM_isna',
 '5_EXENUM_isna',
 '5_brand_num_isna',
 '5_cancel_dum_isna',
 '5_certif

In [85]:
features_frame.columns

Index(['gross_income', 'debt_income_rate', 'other_gross_rate',
       'main_gross_rate', 'yea_sub_exsit', 'adjust_gross_income'],
      dtype='object')

In [565]:
#连续
for feature in features:
    print("\""+feature + "_history_percent"+"\","
         )

"so1_history_percent",
"so2_history_percent",
"so3_history_percent",
"so4_history_percent",
"so5_history_percent",
"totalWagesSo1_history_percent",
"totalWagesSo2_history_percent",
"totalWagesSo3_history_percent",
"totalWagesSo5_history_percent",
"totalPaymentSo1_history_percent",
"totalPaymentSo2_history_percent",
"totalPaymentSo3_history_percent",
"totalPaymentSo4_history_percent",
"totalPaymentSo5_history_percent",
"unPaidSocialInsSo1_history_percent",
"unPaidSocialInsSo2_history_percent",
"unPaidSocialInsSo3_history_percent",
"unPaidSocialInsSo4_history_percent",
"unPaidSocialInsSo5_history_percent",


In [568]:
#离散
for feature in features:
    print("\""+feature + "_history_more"+"\","+
          "\""+feature + "_empty"+"\","
         )

"so1_history_more","so1_empty",
"so2_history_more","so2_empty",
"so3_history_more","so3_empty",
"so4_history_more","so4_empty",
"so5_history_more","so5_empty",
"totalWagesSo1_history_more","totalWagesSo1_empty",
"totalWagesSo2_history_more","totalWagesSo2_empty",
"totalWagesSo3_history_more","totalWagesSo3_empty",
"totalWagesSo5_history_more","totalWagesSo5_empty",
"totalPaymentSo1_history_more","totalPaymentSo1_empty",
"totalPaymentSo2_history_more","totalPaymentSo2_empty",
"totalPaymentSo3_history_more","totalPaymentSo3_empty",
"totalPaymentSo4_history_more","totalPaymentSo4_empty",
"totalPaymentSo5_history_more","totalPaymentSo5_empty",
"unPaidSocialInsSo1_history_more","unPaidSocialInsSo1_empty",
"unPaidSocialInsSo2_history_more","unPaidSocialInsSo2_empty",
"unPaidSocialInsSo3_history_more","unPaidSocialInsSo3_empty",
"unPaidSocialInsSo4_history_more","unPaidSocialInsSo4_empty",
"unPaidSocialInsSo5_history_more","unPaidSocialInsSo5_empty",


In [119]:
file_name=input()
with open("./"+file_name+".pkl", 'wb') as fo:
    pickle.dump(features_frame, fo)

isna_features_0


In [ ]:
financial_features_0

In [528]:
entid_record = tax_abnormal[tax_abnormal["entid"] == 6938]

In [43]:
company_base_info

,entid,honor_num,illegad_num,yea_sub,web_dum,ENTTYPE,INDUSTRYPHY,REGCAP,REGCAPCUR,RECCAP,PARNUM,LIMPARNUM,PARFORM,EXENUM,brand_num,cancel_dum,certificate_num,tax_dum
0,122697049,NaN,NaN,NaN,1,10,9,8103057.4,156.0,4700000.0,NaN,NaN,NaN,NaN,NaN,0,4.0,0
1,959830672,NaN,NaN,NaN,1,5,11,38000.0,156.0,1000.0,NaN,NaN,NaN,NaN,NaN,0,2.0,0
2,564421968,NaN,NaN,NaN,0,8,12,5000.0,156.0,5000.0,NaN,NaN,NaN,NaN,NaN,1,NaN,0
3,620868429,NaN,NaN,NaN,0,8,9,10000.0,156.0,10000.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
4,1176966297,NaN,NaN,NaN,0,55,11,3000.0,156.0,3000.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12274,208371641,NaN,NaN,NaN,0,65,5,20.0,156.0,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12275,331374714,NaN,NaN,NaN,0,99,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12276,470500045,NaN,NaN,NaN,0,5,11,50.0,156.0,500.0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
12277,771018416,NaN,NaN,NaN,0,99,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [550]:
company_base_info.yea_sub.value_counts()

0.00000         421
1.00000           4
2.00000           4
10.00000          3
3.00000           2
5.00000           2
100000.00000      1
7.00000           1
0.50000           1
6.00000           1
40.00000          1
20.15000          1
0.10000           1
49.00000          1
168.00000         1
35.00000          1
3.56201           1
15.00000          1
6.25000           1
9.00000           1
30.00000          1
18.00000          1
1.65000           1
2.40000           1
44.30000          1
75.31250          1
0.05200           1
9.30000           1
54.00000          1
100.00000         1
0.60000           1
1200.00000        1
Name: yea_sub, dtype: int64

In [50]:
history

import pandas as pd
import numpy as np
import os
from collections import Counter
import pickle
pd.set_option('max_colwidth',100)
pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',100)
dir_ = "./trainning_data/corporate_attributes/"
files = os.listdir(dir_)
#读取数据，表名作为变量名
for i in files:
    if i[-3:]!='csv':
        continue
    path=dir_+i
    name=i[:-4]
    print(name+',')
    #print(path)
    cmd = "%s=pd.read_csv(path)" %name    #定义该变量要赋的值（此处为i）
    exec(cmd)
    eval("%s" %name)
#social_fee 特征生成 处理方法类似 assetinfo
entids = company_base_info_new.entid
company_ar_socialfee=company_ar_socialfee.iloc[:,:22].drop_duplicates('entid')

features=[ 'so1', 'so2', 'so3', 'so4', 'so5',
       'totalWagesSo1', 'totalWagesSo2', 'totalWagesSo3', 'totalWagesSo5',
       'totalPaymentSo1', 'totalPaymentSo2', 'totalPaymentSo3',
       'totalPaymentSo4', 'totalPaymentSo5', 'unPaidSocialInsSo1',
       'unPaidSocialInsSo2', 'unPaidSocialInsSo3', 'unPaidSocialInsSo4',
       'unPa

In [ ]:
company_base_info